<h1>                    Model Application</h1>

### Importing Libraries

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from sklearn.linear_model import SGDClassifier
import seaborn as sns
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor 
from mlxtend.plotting import plot_linear_regression
from sklearn.ensemble import RandomForestClassifier
 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [33]:
df=pd.read_csv("xtrain.csv")
df['question1'] = df['question1'].apply(lambda x: str(x))## converting other object types to str while using BOW
df['question2'] = df['question2'].apply(lambda x: str(x))

<h3>Train-Test split</h3>

In [34]:
X_train, X_test, y_train, y_test = train_test_split(df, df['is_duplicate'], test_size=0.25,random_state=0)

In [35]:
X_train.head(5)

,Unnamed: 0,Unnamed: 0.1,id,qid1,qid2,question1,question2,is_duplicate,q1_wordlen,q2_wordlen,common_words,total_words_union,similarity
2606,2606,2606,2606,5178,5179,link origin account ea sport contact number speak,2nd year cs student average problem solve skil...,0,15,36,5,46,0.000000
57228,57228,57228,57228,21983,79791,learn fast,learn minimal time,1,5,10,3,12,0.133631
8382,8382,8382,8382,16344,16345,think right bring child mess world,justified afraid bring child world,1,15,13,5,23,0.000000
60281,60281,60281,60281,105189,105447,german shepherd border collie mix look like,temperament border collie german shepherd mix,0,9,10,0,18,0.285714
48432,48432,48432,48432,86356,86357,year old work software engineer automotive fir...,old senior software engineer large tech compan...,0,34,21,4,51,0.166667


<h3> Removing the ids and extra columns that dont add value in helping the model</h3>

In [36]:
X_train=X_train.drop(["id","Unnamed: 0","Unnamed: 0.1","is_duplicate","qid1","qid2"],axis=1)##while splitting extra columns got added so we remove it
X_test=X_test.drop(["id","Unnamed: 0","Unnamed: 0.1","is_duplicate","qid1","qid2"],axis=1)


In [37]:
X_train,X_test

(                                               question1  \
 2606   link origin account ea sport contact number speak   
 57228                                         learn fast   
 8382                  think right bring child mess world   
 60281        german shepherd border collie mix look like   
 48432  year old work software engineer automotive fir...   
 ...                                                  ...   
 21243   property tax rate granville ohio compare georgia   
 45891                                      control anger   
 42613               stephen hawk work tv big bang theory   
 43567                             video prove time video   
 68268                              wake morning no place   
 
                                                question2  q1_wordlen  \
 2606   2nd year cs student average problem solve skil...          15   
 57228                                 learn minimal time           5   
 8382                  justified afraid bring c

<h2>Vectorizing the text data by bag of words</h2>

In [38]:
t=list(X_train["question1"])

vectorizer_bow_essay = CountVectorizer(min_df=10,max_features=5000)  #selecting top 5000 features
vectorizer_bow_essay.fit(X_train["question1"].astype('U').values)

q1_one_hot_train= vectorizer_bow_essay.transform(t)
q1_one_hot_test=vectorizer_bow_essay.transform(X_test["question1"])


print(q1_one_hot_train.shape,q1_one_hot_test.shape)##printing the shape check 

(75000, 4430) (25000, 4430)


In [39]:
t1=list(X_train["question2"])

vectorizer_bow_essay = CountVectorizer(min_df=1,max_features=5000)  #selecting top 5000 features
vectorizer_bow_essay.fit(t1)

q2_one_hot_train= vectorizer_bow_essay.transform(t1)
q2_one_hot_test=vectorizer_bow_essay.transform(X_test["question2"])

print(q2_one_hot_train.shape)

(75000, 5000)


<h2>Normalizing the numerical features</h2>

In [40]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
normalizer.fit(X_train['q1_wordlen'].values.reshape(1,-1))## since it a 1D array we have to convert to a matrix

q1_wordlen_train = normalizer.transform(X_train['q1_wordlen'].values.reshape(1,-1))
q1_wordlen_test = normalizer.transform(X_test['q1_wordlen'].values.reshape(1,-1))

q1_wordlen_train=q1_wordlen_train.T
q1_wordlen_test=q1_wordlen_test.T


In [41]:
normalizer = Normalizer()
normalizer.fit(X_train['q2_wordlen'].values.reshape(1,-1))

q2_wordlen_train = normalizer.transform(X_train['q2_wordlen'].values.reshape(1,-1))
q2_wordlen_test = normalizer.transform(X_test['q2_wordlen'].values.reshape(1,-1))

q2_wordlen_train=q2_wordlen_train.T
q2_wordlen_test=q2_wordlen_test.T


In [42]:
normalizer = Normalizer()
normalizer.fit(X_train['common_words'].values.reshape(1,-1))

common_words_train = normalizer.transform(X_train['common_words'].values.reshape(1,-1))
common_words_test = normalizer.transform(X_test['common_words'].values.reshape(1,-1))

common_words_train=common_words_train.T
common_words_test=common_words_test.T

In [43]:
normalizer = Normalizer()
normalizer.fit(X_train['total_words_union'].values.reshape(1,-1))

total_words_union_train = normalizer.transform(X_train['total_words_union'].values.reshape(1,-1))
total_words_union_test = normalizer.transform(X_test['total_words_union'].values.reshape(1,-1))

total_words_union_train=total_words_union_train.T
total_words_union_test=total_words_union_test.T

In [44]:
normalizer = Normalizer()
normalizer.fit(X_train['similarity'].values.reshape(1,-1))

similarity_train = normalizer.transform(X_train['similarity'].values.reshape(1,-1))
similarity_test = normalizer.transform(X_test['similarity'].values.reshape(1,-1))

similarity_train=similarity_train.T
similarity_test=similarity_test.T

In [45]:
print(q2_wordlen_train.shape)
print(q2_wordlen_test.shape)

(75000, 1)
(25000, 1)


<h2>Stacking the vectorized and normalized data</h2>

In [46]:
## The datasets will be in csr after stacking 
X_tr=hstack((total_words_union_train,common_words_train,q2_wordlen_train,q1_wordlen_train,q2_one_hot_train,q1_one_hot_train,similarity_train))
X_te=hstack((total_words_union_test,common_words_test,q2_wordlen_test,q1_wordlen_test,q2_one_hot_test,q1_one_hot_test,similarity_test))

<h2>Applying logistic regression</h2>

<h3>Hyperparamenter tuning</h3>

In [47]:
clf=LogisticRegression(penalty="l2",C=0.1)
clf.fit(X_tr,y_train)

parameters = [{'C': [0.1,0.001,0.0001,1, 10, 100, 1000], 'penalty':['l1', 'l2', 'elasticnet', 'none']}]

grid_search=GridSearchCV(estimator = clf,
                           param_grid = parameters,
                           scoring = 'neg_log_loss',
                           cv = 5,
                           verbose=20,## adding verbose just to know our progress
                           n_jobs = -1)
grid_search=grid_search.fit(X_tr, y_train)


Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

In [48]:
grid_search.best_params_

{'C': 0.1, 'penalty': 'l2'}

In [49]:
grid_search.best_score_## in grid search it tries to maximize so we put scoring as negative log loss 
## so the answer we want is negative of the answer it displays

-0.5532082437754017

In [50]:
clf1=LogisticRegression(penalty="l2",C=0.1)
clf1.fit(X_tr,y_train)
    
sig_clf = CalibratedClassifierCV(clf1, method="isotonic")## we are using clibratedclassifier because we need extact probabily of the class it belongs to 
sig_clf.fit(X_tr, y_train)

predict_y =sig_clf.predict_proba(X_te)
print(log_loss(y_test, predict_y))

0.5483662141955461


In [51]:
clf1=LogisticRegression(penalty="l2",C=0.1)
clf1.fit(X_tr,y_train)

predict_y =clf.predict_proba(X_te)
print(log_loss(y_test, predict_y))

0.5495825815986446


#### There is some difference in log loss values in the above implementaion of test dataset.This will be more has the dataset size increases